# App del usuario

Este script simular ser la interfaz del usuario una vez inicia sesión. Le indicará al usuario los pokemos que aparecen a menos de una cierta distancia (que típicamente se fija con un zoom en un mapa, aunque aquí fijaremos a mano). Podemos duplicar el script para simular un segundo usuario. Como el Group ID es aleatorio, ambos usuarios verán todos los mensajes del script ya que deberán comprobar la distancia a la que aparecen todos los pokemons para cada usuario en particular.

In [ ]:
from confluent_kafka import Consumer, KafkaError, TopicPartition, OFFSET_BEGINNING, OFFSET_END
import time
import json
import uuid
import random

from ejercicios.pokemons import TOPIC_POKEMONS, COORDINATES

In [ ]:
USER_LAT = random.gauss(**COORDINATES['GAUSS_LAT_MADRID'])
USER_LON = random.gauss(**COORDINATES['GAUSS_LON_MADRID'])
USER_ID = str(uuid.uuid4())[:6]
MAX_DIST = 50 # en KM
print('User {} in {:.4f},{:.4f}'.format(USER_ID, USER_LAT, USER_LON))

In [ ]:
from math import radians, cos, sin, atan2, sqrt
def get_distance(lat_a, lon_a, lat_b, lon_b):
    radius = 6371 # in km
    # convert degrees to radians
    lat_a, lon_a, lat_b, lon_b = map(radians, [lat_a, lon_a, lat_b, lon_b])
    dist_lon = lon_b - lon_a
    dist_lat = lat_b - lat_a
    a = sin(dist_lat / 2)**2 + cos(lat_a) * cos(lat_b) * sin(dist_lon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = radius * c
    return abs(round(distance, 2))

get_distance(40.3222,-3.5767,40.442550576, -3.132371)

In [ ]:
c = Consumer({
    'bootstrap.servers': 'localhost:9092',
    'group.id': USER_ID,
    'auto.offset.reset': 'earliest'
})
c.subscribe([TOPIC_POKEMONS])

In [ ]:
topic_list = c.list_topics(TOPIC_POKEMONS)
topic_metadata = topic_list.topics[TOPIC_POKEMONS]

In [ ]:
topic_metadata.partitions

In [ ]:
msg = c.poll(0)
print(msg)

Podríamos comprobar sólo los pokemons que aparecen a partir del momento de iniciar sesión, pero vamos a comprobar todos los pokemons que han aparecido desde el origen del topic (o lo que es lo mismo, todos los mensajes que no han sido purgados por expirar su tiempo de retención).

¿Qué ocurriría si usamos `OFFSET_END`?

In [ ]:
try:
    for p in topic_metadata.partitions.keys():
        partition_metadata = c.position([TopicPartition(TOPIC_POKEMONS, p)])
        print('Partition is {}'.format(partition_metadata))
        print('Seeking to 0 on partition {}'.format(p))
        c.seek(TopicPartition(TOPIC_POKEMONS, p, OFFSET_BEGINNING))
except Exception as e:
    print('Error on seek: {}'.format(str(e)))

In [ ]:
while True:
    msg = c.poll(5.0)

    if msg is None:
        continue

    if msg.error():
        print("Consumer error: {}".format(msg.error()))
        continue

    if msg.topic() == TOPIC_POKEMONS:
        pokemon = json.loads(msg.value())
        dist_to_pokemon = get_distance(USER_LAT, USER_LON, pokemon['lat'], pokemon['lon'])
        if dist_to_pokemon < MAX_DIST:
            print('Pokemon {} appears at {:.2f}'.format(pokemon['Name'], dist_to_pokemon))
    
c.close()